# 基于 ViT 的多模态医学图像分割入门教程

> 目标：用 **Vision Transformer (ViT)** 做医学分割，并融合结构化元数据。

流程与 ResNet 版本一致：
1. 准备数据
2. 定义数据集
3. 构建模型（ViT 编码器 + 解码器）
4. 训练
5. 评估


## 0. 环境准备（只需运行一次）

```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
pip install timm numpy pandas pillow matplotlib tqdm
```


## 1. 项目与数据结构

与 ResNet 版本一致：
```
examples/transformer_tutorial2/
  vit_multimodal_seg.ipynb
  data/
    vit_tutorial/
      images/
      masks/
      metadata.csv
```

下面先生成一个**可跑通的小数据集**。


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw

ROOT = Path('examples/transformer_tutorial2/data/vit_tutorial')
IMG_DIR = ROOT / 'images'
MSK_DIR = ROOT / 'masks'
ROOT.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)
MSK_DIR.mkdir(parents=True, exist_ok=True)

rows = []
for i in range(20):
    img = Image.new('L', (224, 224), color=0)
    mask = Image.new('L', (224, 224), color=0)

    draw_img = ImageDraw.Draw(img)
    draw_msk = ImageDraw.Draw(mask)

    cx, cy = np.random.randint(50, 174, size=2)
    r = np.random.randint(15, 40)
    bbox = (cx - r, cy - r, cx + r, cy + r)
    draw_img.ellipse(bbox, fill=200)
    draw_msk.ellipse(bbox, fill=255)

    image_id = f'sample_{i:03d}'
    img.save(IMG_DIR / f'{image_id}.png')
    mask.save(MSK_DIR / f'{image_id}.png')

    rows.append({
        'image_id': image_id,
        'age': int(np.random.randint(18, 80)),
        'sex': int(np.random.randint(0, 2)),
        'modality': int(np.random.randint(0, 3)),
    })

meta = pd.DataFrame(rows)
meta.to_csv(ROOT / 'metadata.csv', index=False)
meta.head()


## 2. 定义数据集

与 ResNet 版本几乎一致，只是图像大小为 224。


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MultimodalSegDataset(Dataset):
    def __init__(self, root_dir, img_size=224):
        self.root_dir = Path(root_dir)
        self.img_dir = self.root_dir / 'images'
        self.msk_dir = self.root_dir / 'masks'
        self.meta = pd.read_csv(self.root_dir / 'metadata.csv')

        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
        ])

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        image_id = row['image_id']

        # 读取图像（灰度）
        img = Image.open(self.img_dir / f'{image_id}.png').convert('L')
        # 读取 mask（灰度）
        mask = Image.open(self.msk_dir / f'{image_id}.png').convert('L')

        img = self.transform(img)
        mask = self.transform(mask)
        mask = (mask > 0.5).float()

        # 把元数据拼成向量
        meta = torch.tensor([row['age'], row['sex'], row['modality']], dtype=torch.float32)
        meta[0] = meta[0] / 100.0
        meta[2] = meta[2] / 2.0
        return img, mask, meta

DATASET_ROOT = 'examples/transformer_tutorial2/data/vit_tutorial'
dataset = MultimodalSegDataset(DATASET_ROOT)
img, mask, meta = dataset[0]
img.shape, mask.shape, meta


## 3. 构建模型：ViT 编码器 + 解码器（带元数据融合）

思路：
1. ViT 把图像切成 patch，并输出 token。
2. 去掉 CLS token，把剩余 token reshape 回特征图。
3. 注入元数据（MLP -> 与特征通道匹配）。
4. 解码器上采样输出 mask。


In [ ]:
import torch.nn as nn
import timm

class ViTSeg(nn.Module):
    def __init__(self, meta_dim=3):
        super().__init__()
        self.vit = timm.create_model('vit_tiny_patch16_224', pretrained=False)
        self.embed_dim = self.vit.embed_dim

        self.meta_mlp = nn.Sequential(
            nn.Linear(meta_dim, 64), nn.ReLU(),
            nn.Linear(64, self.embed_dim)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(self.embed_dim, 256, 2, 2),  # 28x28
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 2, 2),  # 56x56
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 2, 2),   # 112x112
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 2, 2),    # 224x224
            nn.ReLU(),
            nn.Conv2d(32, 1, kernel_size=1)
        )

    def forward(self, x, meta):
        # forward_features 输出 [B, N+1, C] (含 CLS token)
        feats = self.vit.forward_features(x)
        tokens = feats[:, 1:, :]  # 去掉 CLS token
        b, n, c = tokens.shape
        h = w = int(n ** 0.5)
        feat_map = tokens.transpose(1, 2).reshape(b, c, h, w)

        meta_embed = self.meta_mlp(meta).unsqueeze(-1).unsqueeze(-1)
        feat_map = feat_map + meta_embed

        logits = self.decoder(feat_map)
        return logits

model = ViTSeg()
model(img.unsqueeze(0), meta.unsqueeze(0)).shape


## 3.1 快速运行自检（确保模型能前向）

这一小段不会训练，只做**快速检查**：
- 模型能前向
- 输出尺寸正确


In [ ]:
# ✅ 快速自检：模型前向 + 输出尺寸
img, mask, meta = dataset[0]
model = ViTSeg()
logits = model(img.unsqueeze(0), meta.unsqueeze(0))
print('logits shape:', logits.shape)
assert logits.shape[-2:] == mask.shape[-2:], '输出尺寸应与 mask 一致'


## 4. 训练准备（损失 + 指标）


In [ ]:
from torch.optim import Adam

def dice_loss(logits, targets, eps=1e-6):
    probs = torch.sigmoid(logits)
    num = 2 * (probs * targets).sum(dim=(1,2,3))
    den = probs.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3)) + eps
    return 1 - (num / den).mean()

def iou_score(logits, targets, eps=1e-6):
    probs = (torch.sigmoid(logits) > 0.5).float()
    inter = (probs * targets).sum(dim=(1,2,3))
    union = (probs + targets - probs*targets).sum(dim=(1,2,3))
    return ((inter + eps) / (union + eps)).mean().item()

loader = DataLoader(dataset, batch_size=2, shuffle=True)
model = ViTSeg()
optimizer = Adam(model.parameters(), lr=1e-3)


## 5. 训练循环（最小可运行版）


In [ ]:
from tqdm import tqdm

model.train()
for epoch in range(2):
    epoch_loss = 0
    for imgs, masks, metas in tqdm(loader, desc=f'Epoch {epoch+1}'):
        # 前向传播
        logits = model(imgs, metas)
        # 计算损失（BCE + Dice）
        loss = nn.functional.binary_cross_entropy_with_logits(logits, masks) + dice_loss(logits, masks)

        # 反向传播与更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print('avg loss:', epoch_loss / len(loader))


## 6. 评估与可视化


In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    img, mask, meta = dataset[0]
    pred = torch.sigmoid(model(img.unsqueeze(0), meta.unsqueeze(0)))[0,0]

    plt.figure(figsize=(8,3))
    plt.subplot(1,3,1); plt.title('Image'); plt.imshow(img[0], cmap='gray')
    plt.subplot(1,3,2); plt.title('GT Mask'); plt.imshow(mask[0], cmap='gray')
    plt.subplot(1,3,3); plt.title('Pred'); plt.imshow(pred, cmap='gray')
    plt.show()

    print('IoU:', iou_score(pred.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0)))


## 7. 用你自己的数据怎么做？

与 ResNet 版本相同：
1. 替换 `images/` 和 `masks/`。
2. 更新 `metadata.csv`。
3. 如果图像大小不是 224，可以修改 `img_size` 和 ViT 模型（patch size 必须整除图像大小）。

### 预训练/微调建议
- 可以先用大数据训练 ViT，再在你的数据上微调。
- 微调时可冻结前几层，减少过拟合。
